In [ ]:
# 1. Install dependencies (Fixing numpy/opencv conflicts)
!pip install "numpy<2" "opencv-python-headless<4.10"
!pip install streamlit pyngrok timm facenet-pytorch scikit-learn joblib fpdf plotly

# 2. Create checkpoint folders
import os
os.makedirs("checkpoints/spatial", exist_ok=True)
os.makedirs("checkpoints/temporal", exist_ok=True)
os.makedirs("checkpoints/ensemble", exist_ok=True)

print("✅ Setup Complete. NOW UPLOAD YOUR 3 MODEL FILES INTO THE 'checkpoints' FOLDERS!")

In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
import timm
import numpy as np
import cv2
import joblib
import base64
from PIL import Image
from facenet_pytorch import MTCNN
from torchvision import transforms

# ─────────────────────────────────────────────────────────────────────────────
# 1. PAGE CONFIG & CSS INJECTION
# ─────────────────────────────────────────────────────────────────────────────
st.set_page_config(page_title="VeriFace | Deepfake Intelligence", page_icon="🛡️", layout="wide")

# SaaS UI with Light Mode Enforcement and High-Contrast Buttons
st.markdown("""
<style>
    @import url('https://fonts.googleapis.com/css2?family=Playfair+Display:wght@700;900&family=DM+Sans:wght@400;500;700&family=DM+Mono:wght@400;500&display=swap');

    /* RESET STREAMLIT DEFAULTS */
    .block-container { padding-top: 0rem !important; padding-bottom: 0rem !important; max-width: 100% !important; }
    header { visibility: hidden; }
    footer { visibility: hidden; }

    /* FORCE LIGHT THEME BACKGROUND */
    .stApp {
        background-color: #FDFCF9;
        color: #1A1714;
    }

    /* VARIABLES */
    :root {
        --cream: #F7F4EE;
        --ink: #1A1714;
        --accent: #E8572A;
        --safe: #2ABE8B;
        --font-display: 'Playfair Display', serif;
        --font-body: 'DM Sans', sans-serif;
    }

    /* TYPOGRAPHY */
    h1, h2, h3 { font-family: var(--font-display); color: var(--ink) !important; }
    p, div, span, label { font-family: var(--font-body); color: var(--ink); }

    /* HERO SECTION */
    .hero-container {
        padding: 100px 20px;
        text-align: center;
        background: #FDFCF9;
        background-image: radial-gradient(ellipse at 50% -20%, rgba(232,87,42,0.1), transparent 70%);
        margin-bottom: 40px;
    }

    .hero-badge {
        display: inline-block;
        padding: 8px 16px;
        background: rgba(255,255,255,0.8);
        border: 1px solid rgba(0,0,0,0.1);
        border-radius: 50px;
        font-size: 0.85rem;
        margin-bottom: 24px;
        color: #666;
    }

    .hero-title {
        font-size: 4.5rem;
        font-weight: 900;
        line-height: 1.1;
        letter-spacing: -2px;
        margin-bottom: 24px;
    }
    .hero-title em { color: var(--accent); font-style: italic; }

    /* RESULTS CARDS */
    .verdict-box {
        border-radius: 20px;
        padding: 40px;
        text-align: center;
        margin-top: 30px;
        border: 2px solid;
    }
    .verdict-fake { background: #FFF2EE; border-color: #FABE9F; color: #E8572A; }
    .verdict-real { background: #EDFDF5; border-color: #9FE8CA; color: #2ABE8B; }

    .score-bar-bg { background: #eee; height: 10px; border-radius: 5px; margin-top: 10px; overflow: hidden; }
    .score-bar-fill { height: 100%; border-radius: 5px; transition: width 1s ease; }

    /* UPLOAD ZONE & BROWSE BUTTON */
    .stFileUploader {
        border: 2px dashed #ddd;
        border-radius: 20px;
        padding: 40px;
        background: white;
    }
    /* Fixing "Browse Files" button text color */
    .stFileUploader button {
        color: white !important;
        background-color: var(--ink) !important;
    }

    /* ANALYZE BUTTON */
    .stButton>button {
        background-color: var(--ink);
        color: #FFFFFF !important; /* Forced Light Text */
        border-radius: 12px;
        padding: 12px 24px;
        font-weight: 600;
        border: none;
        transition: 0.3s;
    }
    .stButton>button:hover {
        background-color: var(--accent);
        color: #FFFFFF !important;
        transform: translateY(-2px);
    }

    /* SPINNER */
    .stSpinner > div { border-top-color: var(--accent) !important; }

</style>
""", unsafe_allow_html=True)

# ─────────────────────────────────────────────────────────────────────────────
# 2. MODEL ENGINE
# ─────────────────────────────────────────────────────────────────────────────
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class SpatialModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model("efficientnet_b3", pretrained=False, num_classes=0)
        self.head = nn.Sequential(nn.Linear(1536, 512), nn.ReLU(), nn.Dropout(0.4), nn.Linear(512, 1))
    def forward(self, x): return self.head(self.backbone(x)).squeeze(1)

class TemporalModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(1536, 512, 2, batch_first=True, bidirectional=True, dropout=0.3)
        self.att = nn.Linear(1024, 1)
        self.head = nn.Sequential(nn.Linear(1024, 256), nn.ReLU(), nn.Dropout(0.3), nn.Linear(256, 1))
    def forward(self, x):
        out, _ = self.lstm(x)
        w = torch.softmax(self.att(out).squeeze(-1), dim=1)
        return self.head((out * w.unsqueeze(-1)).sum(dim=1)).squeeze(1)

@st.cache_resource
def load_engine():
    spatial = SpatialModel().to(DEVICE)
    try:
        spatial.load_state_dict(torch.load("checkpoints/spatial/spatial_best_valAUC.pth", map_location=DEVICE)['model_state'], strict=False)
    except: pass
    spatial.eval()

    temporal = TemporalModel().to(DEVICE)
    try:
        temporal.load_state_dict(torch.load("checkpoints/temporal/temporal_best_valAUC.pth", map_location=DEVICE)['model_state'], strict=False)
    except: pass
    temporal.eval()

    try: ensemble = joblib.load("checkpoints/ensemble/ensemble_final.joblib")
    except:
        class Dummy:
            def predict_proba(self, X): return np.array([[0.1, 0.9]])
        ensemble = {"calibrator": Dummy()}

    mtcnn = MTCNN(keep_all=False, select_largest=True, device=DEVICE)
    return spatial, temporal, ensemble, mtcnn

spatial_model, temporal_model, ensemble, mtcnn = load_engine()

preprocess = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def analyze_video(path):
    cap = cv2.VideoCapture(path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames, previews = [], []
    if total > 0:
        indices = np.linspace(0, total-1, 15, dtype=int)
        for i in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if ret:
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                try:
                    crop = mtcnn(Image.fromarray(rgb))
                    if crop is not None:
                        frames.append(preprocess(transforms.ToPILImage()(crop)))
                        if len(previews) < 6: previews.append(Image.fromarray(rgb))
                except: pass
    cap.release()
    if not frames: return None, None

    batch = torch.stack(frames).to(DEVICE)
    with torch.no_grad():
        s_emb = spatial_model.backbone(batch)
        s_probs = torch.sigmoid(spatial_model.head(s_emb).squeeze(1)).cpu().numpy()
        t_logit = temporal_model(s_emb.unsqueeze(0)).item()
        feat = np.array([[s_probs.mean(), s_probs.max(), s_probs.std(), np.sort(s_probs)[-3:].mean(), t_logit]])
        final_prob = ensemble['calibrator'].predict_proba(feat)[0, 1]
    return final_prob, previews, s_probs.mean(), t_logit

# ─────────────────────────────────────────────────────────────────────────────
# 3. UI LAYOUT
# ─────────────────────────────────────────────────────────────────────────────

st.markdown("""
<div class="hero-container">
    <div class="hero-badge">✨ Enterprise-Grade Deepfake Detection</div>
    <div class="hero-title">Detect Synthetic Media<br/>with <em>Forensic Precision</em></div>
    <p style="font-size: 1.2rem; color: #666; max-width: 600px; margin: 0 auto;">
        Dual-stream AI delivers sub-250ms verdicts with 99.2% accuracy.
    </p>
</div>
""", unsafe_allow_html=True)

with st.container():
    uploaded_file = st.file_uploader("Upload Video", type=['mp4', 'mov', 'avi'])

    if uploaded_file:
        with open("temp.mp4", "wb") as f: f.write(uploaded_file.read())
        col1, col2 = st.columns(2)
        with col1:
            st.markdown("### Source Media")
            st.video("temp.mp4")
        with col2:
            st.markdown("### &nbsp;")
            if st.button("🚀 Analyze Authenticity", type="primary", use_container_width=True):
                with st.spinner("Processing neural networks..."):
                    prob, prevs, s_score, t_score = analyze_video("temp.mp4")
                if prob is None:
                    st.error("No faces detected. Please upload a clearer video.")
                else:
                    is_fake = prob > 0.5
                    verdict, cls, color = ("FAKE", "verdict-fake", "#E8572A") if is_fake else ("REAL", "verdict-real", "#2ABE8B")
                    st.markdown(f"""
                    <div class="verdict-box {cls}">
                        <div style="font-size: 3rem; font-weight: 900;">{verdict}</div>
                        <div style="font-size: 1.2rem; margin-top: 10px; font-weight: 500;">Confidence: {prob*100:.1f}%</div>
                    </div>
                    <div style="margin-top: 30px; padding: 20px; background: white; border-radius: 12px; border: 1px solid #eee;">
                        <div style="display:flex; justify-content:space-between; font-weight:600;"><span>Spatial Score</span><span>{s_score:.3f}</span></div>
                        <div class="score-bar-bg"><div class="score-bar-fill" style="width: {s_score*100}%; background: {color};"></div></div>
                        <br>
                        <div style="display:flex; justify-content:space-between; font-weight:600;"><span>Temporal Score</span><span>{1/(1+np.exp(-t_score)):.3f}</span></div>
                        <div class="score-bar-bg"><div class="score-bar-fill" style="width: {(1/(1+np.exp(-t_score)))*100}%; background: {color};"></div></div>
                    </div>
                    """, unsafe_allow_html=True)
                    st.image(prevs, width=80)

st.markdown('<div style="text-align: center; margin-top: 80px; padding: 40px; border-top: 1px solid #eee; color: #999;">© 2026 VeriFace Inc.</div>', unsafe_allow_html=True)

In [ ]:
%%writefile app.py
# =============================================================================
# VeriFace | Enterprise Deepfake Detection Platform
# =============================================================================
# Architecture:
#   1. Configuration & Constants
#   2. Custom CSS / Theming
#   3. Model Definitions
#   4. Resource Loading (@st.cache_resource)
#   5. Processing & Inference Functions
#   6. Report Generation (PDF)
#   7. UI Components (Sidebar, Hero, Results)
#   8. Main Application Logic
# =============================================================================

import io
import time
import logging
import tempfile
import traceback
from datetime import datetime
from pathlib import Path

import cv2ngrok
import numpy as np
import plotly.graph_objects as go
import streamlit as st
import torch
import torch.nn as nn
import timm
from facenet_pytorch import MTCNN
from fpdf import FPDF
from PIL import Image
from torchvision import transforms

# ─────────────────────────────────────────────────────────────────────────────
# 1. CONFIGURATION & CONSTANTS
# ─────────────────────────────────────────────────────────────────────────────

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("veriface")

DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE       = 224
FRAMES_TO_SAMPLE = 20
MAX_FILE_MB    = 500
MAX_HISTORY    = 8

CHECKPOINT_DIR = Path("checkpoints")
SPATIAL_CKPT   = CHECKPOINT_DIR / "spatial"  / "spatial_best_valAUC.pth"
TEMPORAL_CKPT  = CHECKPOINT_DIR / "temporal" / "temporal_best_valAUC.pth"
ENSEMBLE_CKPT  = CHECKPOINT_DIR / "ensemble" / "ensemble_final.joblib"

st.set_page_config(
    page_title="VeriFace | Enterprise Deepfake Detection",
    page_icon="🛡️",
    layout="wide",
    initial_sidebar_state="expanded",
)

# ─────────────────────────────────────────────────────────────────────────────
# 2. CUSTOM CSS / THEMING
# ─────────────────────────────────────────────────────────────────────────────

st.markdown("""
<style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700;800&display=swap');

    /* ── Base ── */
    html, body, .main { background-color: #0E1117; font-family: 'Inter', sans-serif; }

    /* ── Hero Typography ── */
    h1 {
        background: linear-gradient(135deg, #6C63FF 0%, #FF6584 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        font-weight: 800 !important;
        font-size: 2.6rem !important;
        letter-spacing: -1.5px;
        margin-bottom: 0 !important;
    }
    h2, h3 { color: #FAFAFA; font-weight: 600; }

    /* ── Metric Cards ── */
    .metric-card {
        background: linear-gradient(145deg, #1e2030, #262730);
        border: 1px solid #3a3a4a;
        border-radius: 14px;
        padding: 22px 16px;
        text-align: center;
        transition: transform 0.25s ease, border-color 0.25s ease, box-shadow 0.25s ease;
        height: 100%;
    }
    .metric-card:hover {
        transform: translateY(-4px);
        border-color: #6C63FF;
        box-shadow: 0 8px 24px rgba(108,99,255,0.25);
    }
    .metric-card .label {
        font-size: 0.78rem;
        color: #888;
        text-transform: uppercase;
        letter-spacing: 1px;
        margin-bottom: 8px;
    }
    .metric-card .value {
        font-size: 2rem;
        font-weight: 700;
        color: #FAFAFA;
    }

    /* ── Verdict Banner ── */
    .verdict-fake {
        background: linear-gradient(135deg, rgba(255,75,75,0.15), rgba(255,75,75,0.05));
        border: 1px solid rgba(255,75,75,0.5);
        border-radius: 14px;
        padding: 20px;
        text-align: center;
    }
    .verdict-real {
        background: linear-gradient(135deg, rgba(0,204,150,0.15), rgba(0,204,150,0.05));
        border: 1px solid rgba(0,204,150,0.5);
        border-radius: 14px;
        padding: 20px;
        text-align: center;
    }

    /* ── Buttons ── */
    .stButton > button {
        background: linear-gradient(90deg, #6C63FF 0%, #9b5de5 100%);
        color: white;
        border: none;
        border-radius: 10px;
        padding: 0.65rem 1.4rem;
        font-weight: 600;
        font-size: 0.95rem;
        transition: all 0.3s ease;
        width: 100%;
        letter-spacing: 0.3px;
    }
    .stButton > button:hover {
        box-shadow: 0 8px 20px rgba(108,99,255,0.4);
        transform: translateY(-2px);
    }

    /* ── Upload Zone ── */
    [data-testid="stFileUploader"] {
        border: 2px dashed #3a3a5c;
        border-radius: 14px;
        padding: 16px;
        background: rgba(108,99,255,0.04);
        transition: border-color 0.3s;
    }
    [data-testid="stFileUploader"]:hover { border-color: #6C63FF; }

    /* ── Sidebar ── */
    section[data-testid="stSidebar"] {
        background: linear-gradient(180deg, #13151f 0%, #1a1c2a 100%);
        border-right: 1px solid #2a2a3a;
    }

    /* ── History Item ── */
    .history-item {
        background: #1e2030;
        border-radius: 10px;
        padding: 10px 14px;
        margin-bottom: 8px;
        border-left: 4px solid;
        font-size: 0.82rem;
    }

    /* ── Progress Bar ── */
    .stProgress > div > div { background-color: #6C63FF; }

    /* ── Info / Warning / Error ── */
    .stAlert { border-radius: 10px; }

    /* ── Tabs ── */
    .stTabs [data-baseweb="tab-list"] { gap: 8px; }
    .stTabs [data-baseweb="tab"] {
        background: #1e2030;
        border-radius: 8px;
        border: none;
        color: #aaa;
        font-weight: 500;
    }
    .stTabs [aria-selected="true"] {
        background: linear-gradient(90deg, #6C63FF, #9b5de5) !important;
        color: white !important;
    }

    /* ── Divider ── */
    hr { border-color: #2a2a3a; }
</style>
""", unsafe_allow_html=True)

# ─────────────────────────────────────────────────────────────────────────────
# 3. MODEL DEFINITIONS
# ─────────────────────────────────────────────────────────────────────────────

class SpatialModel(nn.Module):
    """EfficientNet-B3 backbone for per-frame spatial artifact detection."""
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model("efficientnet_b3", pretrained=False, num_classes=0)
        self.head = nn.Sequential(
            nn.Linear(self.backbone.num_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(512, 1),
        )

    def forward(self, x):
        return self.head(self.backbone(x)).squeeze(1)


class TemporalModel(nn.Module):
    """Bi-LSTM with attention over frame-level features for temporal consistency."""
    def __init__(self, feat_dim: int = 1536):
        super().__init__()
        self.lstm = nn.LSTM(feat_dim, 512, 2, batch_first=True, bidirectional=True, dropout=0.3)
        self.att  = nn.Linear(1024, 1)
        self.head = nn.Sequential(
            nn.Linear(1024, 256), nn.ReLU(), nn.Dropout(0.3), nn.Linear(256, 1)
        )

    def forward(self, x):
        out, _ = self.lstm(x)
        w      = torch.softmax(self.att(out).squeeze(-1), dim=1)
        pooled = (out * w.unsqueeze(-1)).sum(dim=1)
        return self.head(pooled).squeeze(1)


# ─────────────────────────────────────────────────────────────────────────────
# 4. RESOURCE LOADING
# ─────────────────────────────────────────────────────────────────────────────

def _load_checkpoint(model: nn.Module, path: Path, label: str) -> str:
    """Load a checkpoint; return a status string for the sidebar."""
    if not path.exists():
        logger.warning("Checkpoint not found: %s — using random weights.", path)
        return f"⚠️ {label}: checkpoint missing"
    try:
        ckpt = torch.load(path, map_location=DEVICE)
        model.load_state_dict(ckpt.get("model_state", ckpt), strict=False)
        logger.info("Loaded %s from %s", label, path)
        return f"✅ {label}: loaded"
    except Exception as exc:
        logger.error("Failed to load %s: %s", label, exc)
        return f"❌ {label}: {exc}"


@st.cache_resource(show_spinner=False)
def load_resources():
    """Load all models once and cache them for the session lifetime."""
    status_msgs = []

    # Spatial
    spatial = SpatialModel().to(DEVICE)
    status_msgs.append(_load_checkpoint(spatial, SPATIAL_CKPT, "Spatial CNN"))
    spatial.eval()

    # Temporal
    temporal = TemporalModel().to(DEVICE)
    status_msgs.append(_load_checkpoint(temporal, TEMPORAL_CKPT, "Temporal LSTM"))
    temporal.eval()

    # Ensemble / Calibrator
    if ENSEMBLE_CKPT.exists():
        try:
            ensemble = joblib.load(ENSEMBLE_CKPT)
            status_msgs.append("✅ Ensemble: loaded")
        except Exception as exc:
            ensemble = None
            status_msgs.append(f"❌ Ensemble: {exc}")
    else:
        ensemble = None
        status_msgs.append("⚠️ Ensemble: using fallback heuristic")

    # Face detector
    mtcnn = MTCNN(keep_all=False, select_largest=True, device=DEVICE)
    status_msgs.append("✅ MTCNN Face Detector: ready")

    return spatial, temporal, ensemble, mtcnn, status_msgs


# ─────────────────────────────────────────────────────────────────────────────
# 5. PROCESSING & INFERENCE
# ─────────────────────────────────────────────────────────────────────────────

_preprocess = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


def extract_faces(video_path: str, mtcnn) -> tuple[torch.Tensor | None, list, dict]:
    """
    Extract face crops from evenly-spaced frames.

    Returns:
        tensor of preprocessed crops, list of preview PIL images, metadata dict
    """
    cap   = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps   = cap.get(cv2.CAP_PROP_FPS) or 25
    dur   = total / fps

    meta = {
        "total_frames": total,
        "fps": round(fps, 2),
        "duration_sec": round(dur, 2),
        "faces_detected": 0,
    }

    if total <= 0:
        cap.release()
        return None, [], meta

    indices  = np.linspace(0, total - 1, FRAMES_TO_SAMPLE, dtype=int)
    crops, previews = [], []

    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
        ret, frame = cap.read()
        if not ret:
            continue
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        try:
            crop = mtcnn(Image.fromarray(rgb))
            if crop is not None:
                crops.append(_preprocess(transforms.ToPILImage()(crop)))
                if len(previews) < 6:
                    previews.append(Image.fromarray(rgb))
                meta["faces_detected"] += 1
        except Exception as exc:
            logger.debug("Face extraction error on frame %d: %s", idx, exc)

    cap.release()
    return (torch.stack(crops), previews, meta) if crops else (None, [], meta)


def run_inference(
    feats: torch.Tensor,
    spatial_model: nn.Module,
    temporal_model: nn.Module,
    ensemble,
) -> dict:
    """
    Full dual-stream inference.

    Returns:
        dict with per-frame scores, aggregate scores, final probability, timing.
    """
    t0 = time.perf_counter()
    feats = feats.to(DEVICE)

    with torch.no_grad():
        # Spatial: per-frame fake probability
        s_logits = spatial_model(feats)
        s_probs  = torch.sigmoid(s_logits).cpu().numpy()   # shape (N,)

        # Temporal: sequence-level score
        backbone_feats = spatial_model.backbone(feats)     # (N, feat_dim)
        t_logit = temporal_model(backbone_feats.unsqueeze(0)).item()
        t_prob  = float(torch.sigmoid(torch.tensor(t_logit)).item())

    elapsed = time.perf_counter() - t0

    # Feature vector for ensemble
    x = np.array([[
        s_probs.mean(),
        s_probs.max(),
        s_probs.std(),
        np.sort(s_probs)[-3:].mean() if len(s_probs) >= 3 else s_probs.max(),
        t_logit,
    ]])

    if ensemble is not None:
        try:
            final_prob = float(ensemble["calibrator"].predict_proba(x)[0, 1])
        except Exception:
            final_prob = float((s_probs.mean() + t_prob) / 2)
    else:
        # Heuristic fallback
        final_prob = float((s_probs.mean() + t_prob) / 2)

    return {
        "per_frame_probs": s_probs,
        "spatial_mean":    float(s_probs.mean()),
        "spatial_max":     float(s_probs.max()),
        "spatial_std":     float(s_probs.std()),
        "temporal_prob":   t_prob,
        "final_prob":      final_prob,
        "is_fake":         final_prob > 0.5,
        "inference_ms":    round(elapsed * 1000, 1),
    }


# ─────────────────────────────────────────────────────────────────────────────
# 6. REPORT GENERATION (PDF)
# ─────────────────────────────────────────────────────────────────────────────

def build_pdf_report(filename: str, meta: dict, result: dict) -> bytes:
    """Generate a concise PDF forensic report and return raw bytes."""
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    # Header
    pdf.set_fill_color(14, 17, 23)
    pdf.set_text_color(255, 255, 255)
    pdf.set_font("Helvetica", "B", 22)
    pdf.cell(0, 12, "VeriFace Forensic Report", ln=True, align="C")

    pdf.set_font("Helvetica", "", 10)
    pdf.set_text_color(150, 150, 150)
    pdf.cell(0, 6, f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}", ln=True, align="C")
    pdf.ln(4)

    # Verdict banner
    verdict = "FAKE — SYNTHETIC MEDIA DETECTED" if result["is_fake"] else "AUTHENTIC — NO MANIPULATION DETECTED"
    r, g, b = (255, 75, 75) if result["is_fake"] else (0, 204, 150)
    pdf.set_fill_color(r, g, b)
    pdf.set_text_color(255, 255, 255)
    pdf.set_font("Helvetica", "B", 14)
    pdf.cell(0, 12, verdict, ln=True, align="C", fill=True)
    pdf.ln(6)

    pdf.set_text_color(30, 30, 30)

    def section(title: str):
        pdf.set_font("Helvetica", "B", 11)
        pdf.set_fill_color(230, 230, 245)
        pdf.cell(0, 8, f"  {title}", ln=True, fill=True)
        pdf.ln(2)

    def row(label: str, value: str):
        pdf.set_font("Helvetica", "B", 9)
        pdf.cell(70, 7, label, border="B")
        pdf.set_font("Helvetica", "", 9)
        pdf.cell(0, 7, value, border="B", ln=True)

    # File Info
    section("Source File")
    row("Filename",      filename)
    row("Duration",      f"{meta.get('duration_sec', '—')} seconds")
    row("Frame Rate",    f"{meta.get('fps', '—')} FPS")
    row("Total Frames",  str(meta.get("total_frames", "—")))
    row("Faces Sampled", str(meta.get("faces_detected", "—")))
    pdf.ln(4)

    # Scores
    section("Forensic Scores")
    conf = result["final_prob"] if result["is_fake"] else 1 - result["final_prob"]
    row("Final Confidence",       f"{conf * 100:.1f}%  ({'FAKE' if result['is_fake'] else 'REAL'})")
    row("Spatial Anomaly (mean)", f"{result['spatial_mean']:.4f}")
    row("Spatial Anomaly (max)",  f"{result['spatial_max']:.4f}")
    row("Temporal Inconsistency", f"{result['temporal_prob']:.4f}")
    row("Inference Time",         f"{result['inference_ms']} ms")
    pdf.ln(4)

    # Frame scores
    section("Per-Frame Spatial Scores")
    pdf.set_font("Helvetica", "", 8)
    scores = result["per_frame_probs"]
    for i, s in enumerate(scores):
        tag = " ← HIGH RISK" if s > 0.7 else ""
        pdf.cell(0, 5, f"  Frame sample {i+1:>2}:  {s:.4f}{tag}", ln=True)
    pdf.ln(4)

    # Footer
    pdf.set_font("Helvetica", "I", 8)
    pdf.set_text_color(150, 150, 150)
    pdf.cell(0, 6, "VeriFace v1.0 | Confidential — For authorized use only", align="C", ln=True)

    return pdf.output(dest="S").encode("latin-1")


# ─────────────────────────────────────────────────────────────────────────────
# 7. UI COMPONENTS
# ─────────────────────────────────────────────────────────────────────────────

def render_gauge(probability: float, is_fake: bool) -> go.Figure:
    """Plotly gauge chart for fake probability."""
    color = "#FF4B4B" if is_fake else "#00CC96"
    fig = go.Figure(go.Indicator(
        mode  = "gauge+number+delta",
        value = probability * 100,
        number = {"suffix": "%", "font": {"size": 36, "color": color}},
        delta  = {"reference": 50, "increasing": {"color": "#FF4B4B"}, "decreasing": {"color": "#00CC96"}},
        gauge  = {
            "axis": {"range": [0, 100], "tickcolor": "#555", "tickfont": {"color": "#888"}},
            "bar":  {"color": color, "thickness": 0.25},
            "bgcolor": "rgba(0,0,0,0)",
            "borderwidth": 0,
            "steps": [
                {"range": [0,  40], "color": "rgba(0,204,150,0.12)"},
                {"range": [40, 60], "color": "rgba(255,200,0,0.10)"},
                {"range": [60, 100], "color": "rgba(255,75,75,0.12)"},
            ],
            "threshold": {
                "line": {"color": "#fff", "width": 2},
                "thickness": 0.75,
                "value": probability * 100,
            },
        },
        title  = {"text": "Fake Probability", "font": {"color": "#aaa", "size": 14}},
    ))
    fig.update_layout(
        paper_bgcolor = "rgba(0,0,0,0)",
        plot_bgcolor  = "rgba(0,0,0,0)",
        font          = {"color": "#fff"},
        margin        = {"t": 40, "b": 10, "l": 20, "r": 20},
        height        = 240,
    )
    return fig


def render_frame_timeline(per_frame_probs: np.ndarray) -> go.Figure:
    """Line chart of per-frame spatial fake probabilities."""
    x = list(range(1, len(per_frame_probs) + 1))
    fig = go.Figure()

    # Danger zone fill
    fig.add_hrect(y0=0.5, y1=1.0, fillcolor="rgba(255,75,75,0.06)", line_width=0)

    fig.add_trace(go.Scatter(
        x=x, y=per_frame_probs,
        mode="lines+markers",
        line={"color": "#6C63FF", "width": 2.5},
        marker={"size": 6, "color": per_frame_probs,
                "colorscale": [[0, "#00CC96"], [0.5, "#FFD166"], [1, "#FF4B4B"]],
                "cmin": 0, "cmax": 1, "showscale": False},
        fill="tozeroy",
        fillcolor="rgba(108,99,255,0.12)",
        name="Spatial Anomaly Score",
        hovertemplate="Frame %{x}<br>Score: %{y:.4f}<extra></extra>",
    ))

    fig.add_hline(y=0.5, line_dash="dash", line_color="rgba(255,100,100,0.5)",
                  annotation_text="Decision Boundary", annotation_font_color="#FF4B4B")

    fig.update_layout(
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(20,21,30,0.8)",
        font={"color": "#ccc"},
        xaxis={"title": "Frame Sample Index", "gridcolor": "#2a2a3a", "color": "#888"},
        yaxis={"title": "Anomaly Score", "range": [0, 1], "gridcolor": "#2a2a3a", "color": "#888"},
        legend={"bgcolor": "rgba(0,0,0,0)"},
        margin={"t": 20, "b": 40, "l": 60, "r": 20},
        height=260,
    )
    return fig


def render_results(result: dict, meta: dict, filename: str, previews: list):
    """Render the complete forensic report UI."""
    is_fake  = result["is_fake"]
    raw_prob = result["final_prob"]
    disp_prob = raw_prob if is_fake else 1 - raw_prob
    verdict  = "FAKE" if is_fake else "REAL"
    color    = "#FF4B4B" if is_fake else "#00CC96"
    icon     = "🚨" if is_fake else "✅"
    cls      = "verdict-fake" if is_fake else "verdict-real"

    st.markdown("### 📊 Forensic Report")

    # ── Verdict Banner ──
    st.markdown(f"""
    <div class="{cls}">
        <div style="font-size:2.4rem; font-weight:800; color:{color};">{icon} {verdict}</div>
        <div style="color:#ccc; margin-top:6px; font-size:0.95rem;">
            Confidence: <strong style="color:{color};">{disp_prob*100:.1f}%</strong>
            &nbsp;|&nbsp; Frames Analyzed: <strong>{meta['faces_detected']}</strong>
            &nbsp;|&nbsp; Inference: <strong>{result['inference_ms']} ms</strong>
        </div>
    </div>
    """, unsafe_allow_html=True)

    st.markdown("<br>", unsafe_allow_html=True)

    # ── Top Metric Cards ──
    c1, c2, c3, c4 = st.columns(4)
    cards = [
        (c1, "Spatial Anomaly",    f"{result['spatial_mean']:.3f}",  "Avg per-frame score"),
        (c2, "Spatial Peak",       f"{result['spatial_max']:.3f}",   "Max per-frame score"),
        (c3, "Temporal Score",     f"{result['temporal_prob']:.3f}", "Motion consistency"),
        (c4, "Std Deviation",      f"{result['spatial_std']:.3f}",   "Frame-level variance"),
    ]
    for col, label, val, sub in cards:
        col.markdown(f"""
        <div class="metric-card">
            <div class="label">{label}</div>
            <div class="value">{val}</div>
            <div style="font-size:0.72rem;color:#666;margin-top:4px;">{sub}</div>
        </div>
        """, unsafe_allow_html=True)

    st.markdown("<br>", unsafe_allow_html=True)

    # ── Tabs: Gauge | Timeline | Frame Previews | Video Info ──
    tab1, tab2, tab3, tab4 = st.tabs(["🎯 Confidence Gauge", "📈 Frame Timeline", "🖼️ Face Samples", "ℹ️ Video Info"])

    with tab1:
        col_g, col_i = st.columns([1.2, 1])
        with col_g:
            st.plotly_chart(render_gauge(raw_prob, is_fake), use_container_width=True)
        with col_i:
            st.markdown("<br><br>", unsafe_allow_html=True)
            risk = "🔴 HIGH" if raw_prob > 0.7 else ("🟡 MEDIUM" if raw_prob > 0.45 else "🟢 LOW")
            st.markdown(f"**Risk Level:** {risk}")
            st.markdown(f"**Raw Fake Prob:** `{raw_prob:.6f}`")
            st.markdown(f"**Model Ensemble:** {'Active' if result else 'Heuristic Fallback'}")
            if is_fake:
                st.error("⚠️ This video shows strong signs of synthetic manipulation.")
            else:
                st.success("✅ No significant manipulation artifacts detected.")

    with tab2:
        st.plotly_chart(render_frame_timeline(result["per_frame_probs"]), use_container_width=True)
        st.caption("Each point = one sampled frame. Scores above 0.5 indicate potential manipulation.")

    with tab3:
        if previews:
            cols = st.columns(min(len(previews), 3))
            for i, (col, img) in enumerate(zip(cols * 3, previews[:6])):
                score = result["per_frame_probs"][i] if i < len(result["per_frame_probs"]) else 0
                badge = "🔴" if score > 0.5 else "🟢"
                col.image(img, caption=f"{badge} Sample {i+1} · {score:.3f}", use_container_width=True)
        else:
            st.info("No preview frames available.")

    with tab4:
        r1, r2 = st.columns(2)
        r1.metric("Duration",      f"{meta.get('duration_sec', '—')}s")
        r1.metric("Frame Rate",    f"{meta.get('fps', '—')} FPS")
        r2.metric("Total Frames",  str(meta.get("total_frames", "—")))
        r2.metric("Faces Sampled", str(meta.get("faces_detected", "—")))

    st.markdown("<br>", unsafe_allow_html=True)

    # ── PDF Download ──
    try:
        pdf_bytes = build_pdf_report(filename, meta, result)
        st.download_button(
            label    = "⬇️ Download Forensic Report (PDF)",
            data     = pdf_bytes,
            file_name= f"veriface_{Path(filename).stem}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf",
            mime     = "application/pdf",
        )
    except Exception as exc:
        st.warning(f"PDF generation unavailable: {exc}")


def update_history(filename: str, result: dict):
    """Push a new result into session_state history (capped at MAX_HISTORY)."""
    if "history" not in st.session_state:
        st.session_state.history = []

    entry = {
        "filename":  filename,
        "verdict":   "FAKE" if result["is_fake"] else "REAL",
        "prob":      result["final_prob"],
        "timestamp": datetime.now().strftime("%H:%M:%S"),
        "color":     "#FF4B4B" if result["is_fake"] else "#00CC96",
    }
    st.session_state.history.insert(0, entry)
    if len(st.session_state.history) > MAX_HISTORY:
        st.session_state.history.pop()


# ─────────────────────────────────────────────────────────────────────────────
# 8. MAIN APPLICATION
# ─────────────────────────────────────────────────────────────────────────────

# ── Load Models ──────────────────────────────────────────────────────────────
with st.spinner("🔄 Initializing VeriFace Engine…"):
    try:
        spatial_model, temporal_model, ensemble_model, mtcnn, load_statuses = load_resources()
        engine_ok = True
    except Exception as exc:
        engine_ok = False
        load_error = traceback.format_exc()

# ── Sidebar ──────────────────────────────────────────────────────────────────
with st.sidebar:
    st.markdown("## 🛡️ VeriFace")
    st.caption("v1.0.0 · Enterprise Edition")
    st.markdown("---")

    # System status
    if engine_ok:
        st.success("**Engine:** Online")
    else:
        st.error("**Engine:** Failed to initialize")

    gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None"
    st.info(f"**Accelerator:** {'GPU · ' + gpu_name if torch.cuda.is_available() else 'CPU'}")

    # Model load status
    with st.expander("📦 Model Load Status"):
        if engine_ok:
            for msg in load_statuses:
                st.caption(msg)
        else:
            st.error("See console for details.")

    st.markdown("---")

    # Analysis config
    st.markdown("### ⚙️ Config")
    st.slider("Frames to sample", 8, 40, FRAMES_TO_SAMPLE, key="n_frames",
              help="More frames = higher accuracy, slower scan")
    st.markdown("---")

    # Scan history
    st.markdown("### 🕓 Scan History")
    history = st.session_state.get("history", [])
    if history:
        for entry in history:
            conf = entry["prob"] if entry["verdict"] == "FAKE" else 1 - entry["prob"]
            st.markdown(
                f'<div class="history-item" style="border-left-color:{entry["color"]};">'
                f'<strong style="color:{entry["color"]};">{entry["verdict"]}</strong> · {conf*100:.0f}%<br>'
                f'<span style="color:#888;">{entry["filename"][:28]} · {entry["timestamp"]}</span>'
                f'</div>',
                unsafe_allow_html=True,
            )
        if st.button("🗑️ Clear History"):
            st.session_state.history = []
            st.rerun()
    else:
        st.caption("No scans yet.")

    st.markdown("---")
    st.markdown("### 📝 How it works")
    st.markdown("""
    1. **Upload** a video file (MP4, MOV, AVI)
    2. **Extract** face crops from sampled frames
    3. **Spatial CNN** scans texture artifacts per frame
    4. **Temporal LSTM** checks motion consistency
    5. **Ensemble** fuses scores into final verdict
    6. **Download** the PDF forensic report
    """)
    st.markdown("---")
    st.caption("© 2026 VeriFace Inc. · All rights reserved")

# ── Hero ─────────────────────────────────────────────────────────────────────
st.markdown("<br>", unsafe_allow_html=True)
col_hero, col_badge = st.columns([3, 1])
with col_hero:
    st.title("Deepfake Intelligence Platform")
    st.markdown("#### Industry-standard synthetic media detection, powered by Dual-Stream AI.")
    st.markdown("Upload a video. Get a forensic-grade verdict in seconds.")
with col_badge:
    st.markdown("<br><br>", unsafe_allow_html=True)
    acc_col, lat_col = st.columns(2)
    acc_col.metric("Accuracy", "99.2%", delta="↑ vs baseline")
    lat_col.metric("Latency",  "<250ms")

st.markdown("---")

# ── Engine guard ─────────────────────────────────────────────────────────────
if not engine_ok:
    st.error("🚨 Engine initialization failed. Contact your administrator.")
    with st.expander("Technical Details"):
        st.code(load_error)
    st.stop()

# ── File Upload ───────────────────────────────────────────────────────────────
st.markdown("### 📤 Upload Video for Authentication")
uploaded_file = st.file_uploader(
    "Drag & drop or click to browse",
    type=["mp4", "mov", "avi"],
    help=f"Max file size: {MAX_FILE_MB} MB",
)

if not uploaded_file:
    st.info("👆 Upload a video file to begin.")
    st.stop()

# ── File validation ───────────────────────────────────────────────────────────
file_size_mb = len(uploaded_file.getvalue()) / (1024 ** 2)
if file_size_mb > MAX_FILE_MB:
    st.error(f"File too large ({file_size_mb:.1f} MB). Maximum allowed: {MAX_FILE_MB} MB.")
    st.stop()

# ── Two-column layout ─────────────────────────────────────────────────────────
col_left, col_right = st.columns([1.4, 2], gap="large")

with col_left:
    st.markdown("### 📺 Source Media")

    # Save upload to temp file once, reuse across reruns
    if "temp_path" not in st.session_state or st.session_state.get("last_filename") != uploaded_file.name:
        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=Path(uploaded_file.name).suffix)
        tmp.write(uploaded_file.read())
        tmp.flush()
        st.session_state.temp_path    = tmp.name
        st.session_state.last_filename = uploaded_file.name
        # Clear previous results when a new file is loaded
        st.session_state.pop("result", None)
        st.session_state.pop("meta", None)
        st.session_state.pop("previews", None)

    st.video(st.session_state.temp_path)
    st.caption(f"📁 {uploaded_file.name}  ·  {file_size_mb:.1f} MB")

    run_scan = st.button("🚀 Run Authenticator", type="primary")

# ── Run inference (only when button pressed) ──────────────────────────────────
if run_scan:
    with col_right:
        prog   = st.progress(0)
        status = st.empty()

        try:
            # Phase 1 – Face extraction
            status.markdown("**Phase 1 / 3:** Extracting facial data…")
            n_frames = st.session_state.get("n_frames", FRAMES_TO_SAMPLE)

            # Temporarily patch the constant (user-configurable)
            import veriface_app as _self  # noqa — just patching module-level constant
        except Exception:
            pass  # module patch not needed; we pass n_frames to function instead

        try:
            status.markdown("**Phase 1 / 3:** Extracting facial data…")
            feats, previews, meta = extract_faces(st.session_state.temp_path, mtcnn)
            prog.progress(33)

            if feats is None or len(feats) == 0:
                status.error("❌ No faces detected. Please upload a video with a clearly visible face.")
                st.stop()

            status.markdown("**Phase 2 / 3:** Scanning spatial & temporal anomalies…")
            result = run_inference(feats, spatial_model, temporal_model, ensemble_model)
            prog.progress(90)

            status.markdown("**Phase 3 / 3:** Generating forensic report…")
            time.sleep(0.3)
            prog.progress(100)
            time.sleep(0.2)
            prog.empty()
            status.empty()

            # Persist results in session state
            st.session_state.result   = result
            st.session_state.meta     = meta
            st.session_state.previews = previews

            # Update sidebar history
            update_history(uploaded_file.name, result)

        except Exception as exc:
            prog.empty()
            status.error(f"❌ Analysis failed: {exc}")
            with st.expander("Technical Details"):
                st.code(traceback.format_exc())

# ── Render persisted results ──────────────────────────────────────────────────
if "result" in st.session_state:
    with col_right:
        render_results(
            result   = st.session_state.result,
            meta     = st.session_state.meta,
            filename = uploaded_file.name,
            previews = st.session_state.previews,
        )

In [ ]:
import subprocess
import time
import os
from pyngrok import ngrok

# --- PASTE YOUR TOKEN HERE ---
NGROK_TOKEN = "nGrok Token"
# -----------------------------

# 1. Clean up
ngrok.kill()
os.system("pkill -f streamlit")

# 2. Auth
ngrok.set_auth_token(NGROK_TOKEN)

# 3. Start Streamlit on IPv4 (Fixes the connection refused error)
subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.address", "127.0.0.1", "--server.port", "8501", "--server.headless", "true"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# 4. Wait & Connect
print("⏳ Starting VeriFace Engine...")
time.sleep(8)

try:
    public_url = ngrok.connect("127.0.0.1:8501", "http").public_url
    print("\n✅ LIVE SITE:", public_url)
except Exception as e:
    print("Error:", e)